In [ ]:
import sys, os 
import ipywidgets as widgets
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent))
from demoTools.demoutils import *
from IPython.display import display

In [ ]:
%%writefile healthcare_job_openvino.sh

cd $PBS_O_WORKDIR
DEVICE=$1
RESULTS=$2

if [ "$DEVICE" = "HETERO:FPGA,CPU" ]; then
    # Environment variables and compilation for edge compute nodes with FPGAs
    source /opt/fpga_support_files/setup_env.sh
    aocl program acl0 /opt/intel/openvino/bitstreams/a10_vision_design_bitstreams/2019R1_PL1_FP11_MobileNet_Clamp.aocx
fi
     
# Running the object detection code
SAMPLEPATH=$PBS_O_WORKDIR
python3 healthcare_openvino.py     -d $DEVICE \
                                   -l /opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_avx2.so \
                                   -r $RESULTS


In [ ]:
job_id_core = !qsub healthcare_job_openvino.sh -l nodes=1:tank-870:i5-6500te  -F "CPU results/"
print(job_id_core[0]) 
if job_id_core:
    progressIndicator('results/'+job_id_core[0], 'i_progress.txt', "Processing", 0, 100)

In [ ]:
job_id_xeon = !qsub healthcare_job_openvino.sh -l nodes=1:tank-870:e3-1268l-v5  -F "CPU results/"
print(job_id_xeon[0]) 
if job_id_xeon:
    progressIndicator('results/'+job_id_xeon[0], 'i_progress.txt', "Inference+Rendering", 0, 100)

In [ ]:
#Submit job to the queue
job_id_gpu = !qsub healthcare_job_openvino.sh -l nodes=1:tank-870:i5-6500te:intel-hd-530 -F "GPU results/"
print(job_id_gpu) 
if job_id_gpu:
    progressIndicator('results/'+job_id_gpu[0], 'i_progress.txt', "Inference+Rendering", 0, 100)

In [ ]:
#Submit job to the queue
job_id_ncs2 = !qsub healthcare_job_openvino.sh -l nodes=1:tank-870:i5-6500te:intel-ncs2 -F "MYRIAD results/"
print(job_id_ncs2[0]) 
if job_id_ncs2:
    progressIndicator('results/'+job_id_ncs2[0], 'i_progress.txt', "Inference+Rendering", 0, 100)

In [ ]:
job_id_hddlr = !qsub healthcare_job_openvino.sh -l nodes=1:tank-870:i5-6500te:iei-mustang-v100-mx8 -F "HDDL results/"
print(job_id_hddlr[0]) 
if job_id_hddlr:
    progressIndicator('results/'+job_id_hddlr[0], 'i_progress.txt', "Inference+Rendering", 0, 100)

In [ ]:
outputHTML('IEI Tank (Intel Core CPU)',
          'results/'+job_id_core[0], '.png')

In [ ]:
outputHTML('IEI Tank Xeon (Intel Xeon CPU)',
          'results/'+job_id_xeon[0], '.png')

In [ ]:
outputHTML('IEI Intel GPU (Intel Core + Onboard GPU)',
          'results/'+job_id_gpu[0], '.png')

In [ ]:
outputHTML('IEI Tank + Intel CPU + Intel NCS2',
          'results/'+job_id_ncs2[0], '.png')

In [ ]:
outputHTML('IEI Tank + IEI Mustang-V100-MX8 ( Intel® Movidius™ Myriad™ X Vision Processing Unit (VPU))',
          'results/'+job_id_hddlr[0], '.png')

In [ ]:
outputHTML('IEI Tank + IEI Mustang-F100-A10 (Intel® Arria® 10 FPGA)',
          'results/'+job_id_fpga[0], '.png')

In [ ]:
outputHTML('UP Squared Grove IoT Development Kit (UP2)',
          'results/'+job_id_up2[0], '.png')

In [ ]:
arch_list = [('core', 'Intel Core\ni5-6500TE\nCPU'),
             ('xeon', 'Intel Xeon\nE3-1268L v5\nCPU'),
             ('gpu', ' Intel Core\ni5-6500TE\nGPU'),
             ('fpga', ' IEI Mustang\nF100-A10\nFPGA'),
             ('ncs2', 'Intel\nNCS2'),
             ('vpu', ' IEI Mustang\nV100-MX8\nVPU'),
             ('up2', 'Intel Atom\nx7-E3950\nUP2/GPU')]

stats_list = []
for arch, a_name in arch_list:
    if 'job_id_'+arch in vars():
        stats_list.append(('results/'+vars()['job_id_'+arch][0]+'/stats.txt', a_name))
    else:
        stats_list.append(('placeholder'+arch, a_name))

summaryPlot(stats_list, 'Architecture', 'Time, seconds', 'Inference Engine Processing Time', 'time' )

summaryPlot(stats_list, 'Architecture', 'Frames per second', 'Inference Engine FPS', 'fps' )